# Day2-数据读取与分析
　　我们在实现具体的任务前，先对数据进行分析是很有必要的。可能不注意的人，就是直接读取，接着一顿基本处理，而不考虑数据本身的特点。虽然我自己在看到DataWhale给的分析前，我也是这么想的，但是看到这么一顿分析，心里暗想:卧槽，还能看出这些东西。这才更加意识到，在正式开始写模型前，首先对数据的分布做一些观察，然后选择合适的策略是做预处理，是非常有必要的。
<br>　　对于分类问题，到底要观察什么？ 很重要的一点大概就是数据的分布了。
<br>　　另外，由于这是一个NLP问题，在这个特定的领域中，与文本相关的都需要考虑到这样一个问题:文本的长短是否比较集中。或者说，如何处理文本中的一些少量的超长的文本。
<br>　　对于第一个问题，我们要大致看一下各种类别的数量分布是否比较平衡？如果某些类别数量很少，而某些类别数量很多，且差距非常大的话，直接去训练可能效果就不会太好，则需要我们对数据做一定的处理。
<br>　　对于第二个问题，我们同样需要需要做一些处理，也需要我们去选择具体的处理方式，是通过简单的截断处理，还是通过选择其他的表示形式来处理呢？
<br>　　具体的解决方案，先放在后面，接下来，我们先来看一下DataWhale官方的数据分析的过程。     

# Task2 数据读取与数据分析


## 数据读取与数据分析

本章主要内容为数据读取和数据分析，具体使用`Pandas`库完成数据读取操作，并对赛题数据进行分析构成。

### 学习目标

- 学习使用`Pandas`读取赛题数据
- 分析赛题数据的分布规律

### 数据读取

赛题数据虽然是文本数据，每个新闻是不定长的，但任然使用csv格式进行存储。因此可以直接用`Pandas`完成数据读取的操作。

```python
import pandas as pd
train_df = pd.read_csv('../input/train_set.csv', sep='\t', nrows=100)
```

这里的`read_csv`由三部分构成：

- 读取的文件路径，这里需要根据改成你本地的路径，可以使用相对路径或绝对路径；

- 分隔符`sep`，为每列分割的字符，设置为`\t`即可；
- 读取行数`nrows`，为此次读取文件的函数，是数值类型（由于数据集比较大，建议先设置为100）；


![task2_train_head](https://img-blog.csdnimg.cn/20200714203730739.png)

上图是读取好的数据，是表格的形式。第一列为新闻的类别，第二列为新闻的字符。

### 数据分析

在读取完成数据集后，我们还可以对数据集进行数据分析的操作。虽然对于非结构数据并不需要做很多的数据分析，但通过数据分析还是可以找出一些规律的。



此步骤我们读取了所有的训练集数据，在此我们通过数据分析希望得出以下结论：

- 赛题数据中，新闻文本的长度是多少？
- 赛题数据的类别分布是怎么样的，哪些类别比较多？
- 赛题数据中，字符分布是怎么样的？



#### 句子长度分析

在赛题数据中每行句子的字符使用空格进行隔开，所以可以直接统计单词的个数来得到每个句子的长度。统计并如下：

```python
%pylab inline
train_df['text_len'] = train_df['text'].apply(lambda x: len(x.split(' ')))
print(train_df['text_len'].describe())
```

输出结果为：

```
Populating the interactive namespace from numpy and matplotlib
count    200000.000000
mean        907.207110
std         996.029036
min           2.000000
25%         374.000000
50%         676.000000
75%        1131.000000
max       57921.000000
Name: text_len, dtype: float64
```

对新闻句子的统计可以得出，本次赛题给定的文本比较长，每个句子平均由907个字符构成，最短的句子长度为2，最长的句子长度为57921。

下图将句子长度绘制了直方图，可见大部分句子的长度都几种在2000以内。

```python
_ = plt.hist(train_df['text_len'], bins=200)
plt.xlabel('Text char count')
plt.title("Histogram of char count")
```

![task2_char_hist](https://img-blog.csdnimg.cn/20200714203836905.png)

#### 新闻类别分布

接下来可以对数据集的类别进行分布统计，具体统计每类新闻的样本个数。

```python
train_df['label'].value_counts().plot(kind='bar')
plt.title('News class count')
plt.xlabel("category")
```

![task2_class_hist](https://img-blog.csdnimg.cn/20200714203929296.png)

在数据集中标签的对应的关系如下：{'科技': 0, '股票': 1, '体育': 2, '娱乐': 3, '时政': 4, '社会': 5, '教育': 6, '财经': 7, '家居': 8, '游戏': 9, '房产': 10, '时尚': 11, '彩票': 12, '星座': 13}

从统计结果可以看出，赛题的数据集类别分布存在较为不均匀的情况。在训练集中科技类新闻最多，其次是股票类新闻，最少的新闻是星座新闻。

#### 字符分布统计

接下来可以统计每个字符出现的次数，首先可以将训练集中所有的句子进行拼接进而划分为字符，并统计每个字符的个数。

```python
from collections import Counter
all_lines = ' '.join(list(train_df['text']))
word_count = Counter(all_lines.split(" "))
word_count = sorted(word_count.items(), key=lambda d:d[1], reverse = True)

print(len(word_count))
# 6869

print(word_count[0])
# ('3750', 7482224)

print(word_count[-1])
# ('3133', 1)
```

从统计结果中可以看出，在训练集中总共包括6869个字，其中编号3750的字出现的次数最多，编号3133的字出现的次数最少。

这里还可以根据字在每个句子的出现情况，反推出标点符号。下面代码统计了不同字符在句子中出现的次数，其中字符3750，字符900和字符648在20w新闻的覆盖率接近99%，很有可能是标点符号。

```python
train_df['text_unique'] = train_df['text'].apply(lambda x: ' '.join(list(set(x.split(' ')))))
all_lines = ' '.join(list(train_df['text_unique']))
word_count = Counter(all_lines.split(" "))
word_count = sorted(word_count.items(), key=lambda d:int(d[1]), reverse = True)

print(word_count[0])
# ('3750', 197997)

print(word_count[1])
# ('900', 197653)

print(word_count[2])
# ('648', 191975)
```



### 数据分析的结论

通过上述分析我们可以得出以下结论：

1. 赛题中每个新闻包含的字符个数平均为1000个，还有一些新闻字符较长；
2. 赛题中新闻类别分布不均匀，科技类新闻样本量接近4w，星座类新闻样本量不到1k；
3. 赛题总共包括7000-8000个字符；

通过数据分析，我们还可以得出以下结论：

1. 每个新闻平均字符个数较多，可能需要截断；

2. 由于类别不均衡，会严重影响模型的精度；

### 本章小结

本章对赛题数据进行读取，并新闻句子长度、类别和字符进行了可视化分析。

前面我们提到了，要观察数据的分布情况，那是一个意识上的强调，而这里，则是给出了一个具体的code是怎样写的。供我们参考

在DataWhale的《数据分析的结论》部分中，我们可以看到，其得到了以下两个结论:

1. 每个新闻平均字符个数较多，可能需要截断；

2. 由于类别不均衡，会严重影响模型的精度；
<br><br>
在搜索资料的过程中，我发现了在通常情况下可以考虑的一些解决方案

* [缓解类别不平衡的问题:EDA,即简单数据增强(easy data augmentation)](https://www.worldlink.com.cn/osdir/eda-nlp-for-chinese.html) 

  具体来说，该方案具有以下几个方法

  * 同义词替换(Synonym Replacement, SR)：从句子中随机选取n个不属于停用词集的单词，并随机选择其同义词替换它们

  * 随机插入(Random Insertion, RI)：随机的找出句中某个不属于停用词集的词，并求出其随机的同义词，将该同义词插入句子的一个随机位置。重复n次

  * 随机交换(Random Swap, RS)：随机的选择句中两个单词并交换它们的位置。重复n次；

  * 随机删除(Random Deletion, RD)：以 $p$ 的概率，随机的移除句中的每个单词

    需要说明的是，该作者指出该方案在较小的数据集上能发挥更好性能效果，比较大的数据集上可能并不够好
    
    #### 1. EDA提高文本分类的效果的原理是什么？

    - 生成类似于原始数据的增强数据会引入一定程度的噪声，有助于防止过拟合；

    - 使用EDA可以通过同义词替换和随机插入操作引入新的词汇，允许模型泛化到那些在测试集中但不在训练集中的单词；

    #### 2. 为什么使用EDA而不使用语境增强、噪声、GAN和反向翻译？

    上述的其它增强技术作者都希望你使用，它们确实在一些情况下取得比EDA较好的性能，但是，由于需要一个深度学习模型，这些技术往往在其取得的效果面前，付出的实现代价更高。而EDA的目标在于，使用简单方便的技术就能取得相接近的结果。

    #### 3. EDA是否有可能会降低模型的性能？

    确实有可能。原因在于，EDA有可能在增强的过程中，改变了句子的意思，但其仍保留原始的类别标签，从而产生了标签错误的句子。
    
    
对于匿名数据如何去找同义词，我目前还不太清楚应该怎样做，暂时留个坑吧

对于文本过长，目前可能比较简单粗暴的处理是截断。但是也有一些其他的方案，比如换用对长度限制更好的模型；或者将长句子拆分，这将在之后的具体实现中去尝试。

### 本章作业

1. 假设字符3750，字符900和字符648是句子的标点符号，请分析赛题每篇新闻平均由多少个句子构成？


In [9]:
import numpy as np
import pandas as pd
import re

In [10]:
train_df = pd.read_csv('../input/train_set.csv', sep='\t')

In [11]:
train_df['sents_len'] = train_df['text'].apply(lambda x: len(re.split(' 3750 | 900 | 648 ', x)))
train_df['sents_len'].describe()

count    200000.000000
mean         78.094350
std          84.052108
min           1.000000
25%          27.000000
50%          55.000000
75%         100.000000
max        3351.000000
Name: sents_len, dtype: float64

2. 统计每类新闻中出现次数对多的字符

In [16]:
import numpy as np
import pandas as pd
from collections import Counter

In [17]:
%%time
train_df = pd.read_csv('../input/train_set.csv', sep='\t')
train_df['text'] = train_df['text'].apply(lambda x: x.replace(' 3750',"").replace(' 900',"").replace(' 648',""))

# 同一类的拼接到一起
for i in range(0, 14):
    df = train_df[train_df['label'] == i]['text']
    word_count = Counter(" ".join(df.values.tolist()).split())
    print(i, word_count.most_common(1)[0])  # 新闻类，次数最多的字符及次数

0 ('3370', 501748)
1 ('3370', 625287)
2 ('7399', 351851)
3 ('6122', 187882)
4 ('4411', 120317)
5 ('6122', 159089)
6 ('6248', 193738)
7 ('3370', 158886)
8 ('6122', 57290)
9 ('7328', 46466)
10 ('3370', 67703)
11 ('4939', 18587)
12 ('4464', 51420)
13 ('4939', 9648)
Wall time: 1min 53s
